# Week 8 - Code Samples - [Group name]

## References

## Contents
1. Setup 
1. Read dataset
3. Preprocessing pipeline
4. Estimator pipeline
5. Inspect pipeline attributes
6. Compare `SelectKBest` and `SelectKBestDF`

## 1. Setup

Below libraries are loaded, a helper function is defined, and the features dataframe and target series are created for use by the `GridSearchCV` demonstrations that follow.

Load libraries and display version numbers.

In [7]:
import pandas  as pd
import numpy   as np
import sklearn as sk
print('sklearn',sk.__version__)
print('pandas ',pd.__version__)
print('numpy  ',np.__version__)

sklearn 0.19.2
pandas 0.24.2
numpy 1.16.3

These version numbers may not be the most recent or correspond to the documentation you locate via Google.

The `display_pdf` function displays a pandas dataframe using the databricks display function.

In [10]:
def display_pdf(a_pdf):
  display(spark.createDataFrame(a_pdf))

In [11]:
def est_grid_results_pdf(my_est_grid_obj): 
  import pandas as pd
  import numpy  as np
  return pd.DataFrame(data=my_est_grid_obj.cv_results_) \
           .loc[:,lambda df: np.logical_or(df.columns.str.startswith('param_'),
                                           df.columns.str.endswith('test_score'))
               ]

## 2. Read dataset

In [13]:
%sh ls -hot /dbfs/mnt/group-ma707/data/*

-rw-r--r-- 1 root 259K Jan 29 17:44 /dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv
-rw-r--r-- 1 root 12M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_coal.csv
-rw-r--r-- 1 root 11M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv

In [14]:
%sh head -n 3 /dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv

"Date","BCI","C5","C7","P1A_03","P2A_03","P4_03","P3A~IV","SHFE_AL3","RICI","ICE_KC3","CME_SM3","CME_LC2","OPEC_ORB","SHFE_CU3","CME_LN1","CME_FC3","P3A_03","SHFE_RB3","CME_S2","ICE_SB3","CME_LN3","CME_LN2","ICE_TIB3","ICE_TIB4","BCI_5TC"
2011-12-05,3390,12.529,15.718,15329,24935,3324,16.54,16.54,3688.22,3688.22,3688.22,3688.22,110.3,110.3,110.3,110.3,10503,10503,1136.5,1136.5,1136.5,1136.5,-6.66,-6.6,29966
2011-12-06,3387,12.379,15.705,15328,24967,3288,16.55,16.55,3700.6,3700.6,3700.6,3700.6,109.49,109.49,109.49,109.49,10412,10412,1139.5,1139.5,1139.5,1139.5,-6.84,-6.77,29990

In [15]:
import pandas as pd
bci_pdf = pd.read_csv('/dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv') \
            .rename(columns={'P3A~IV':'P3A_IV'}) \
            .assign(datetime=lambda pdf: pd.to_datetime(pdf.Date)) \
            .drop('Date', axis=1)
bci_pdf.columns = bci_pdf.columns.str.lower()
bci_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602 entries, 0 to 1601
Data columns (total 26 columns):
bci 1602 non-null int64
c5 1602 non-null float64
c7 1602 non-null float64
p1a_03 1602 non-null int64
p2a_03 1602 non-null int64
p4_03 1602 non-null int64
p3a_iv 1602 non-null float64
shfe_al3 1602 non-null float64
rici 1602 non-null float64
ice_kc3 1602 non-null float64
cme_sm3 1602 non-null float64
cme_lc2 1602 non-null float64
opec_orb 1602 non-null float64
shfe_cu3 1602 non-null float64
cme_ln1 1602 non-null float64
cme_fc3 1602 non-null float64
p3a_03 1602 non-null int64
shfe_rb3 1602 non-null int64
cme_s2 1602 non-null float64
ice_sb3 1602 non-null float64
cme_ln3 1602 non-null float64
cme_ln2 1602 non-null float64
ice_tib3 1602 non-null float64
ice_tib4 1602 non-null float64
bci_5tc 1602 non-null int64
datetime 1602 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(18), int64(7)
memory usage: 325.5 KB

The BCI dataset has no missing values. See below:

In [17]:
bci_pdf.isnull().sum().sum()

Out[ 7 ]: 0

## 3. Preprocessing pipeline

In [19]:
%run "./W08.2 Feature creation (inc)"

See [W08.2 Feature creation (inc)](https://bentley.cloud.databricks.com/#notebook/1176459)

Create a preprocessing pipeline to create the feature-target dataset from the `bci_pdf` dataframe.

In [22]:
from sklearn.pipeline      import Pipeline, FeatureUnion

fea_tgt_pipe = Pipeline([
  ('fea_tgt', FeatureUnionDF([('tgt', CreateTargetVarDF(var='bci_5tc')), 
                              ('nam', CreateNamedVarsDF(var_list=['datetime'])),
                              ('lag', CreateLagVarsDF  (var_list=list(set(bci_pdf.columns) - 
                                                                      set(['bci_5tc','datetime'])),
                                                        lag_list=range(3,11))),
                              ('dat', CreateDatetimeVarsDF(var='datetime',
                                                           var_list=['year','month','day','weekofyear','weekday','dayofyear']))
                             ])),
  ('drop_na', DropNaRowsDF())
])

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1251865> in <module> () 
 2 
 3 fea_tgt_pipe = Pipeline([
 ----> 4 ('fea_tgt', FeatureUnionDF([('tgt', CreateTargetVarDF(var='bci_5tc')), 
 5 ( 'nam' , CreateNamedVarsDF ( var_list = [ 'datetime' ] ) ) , 
 6 ('lag', CreateLagVarsDF (var_list=list(set(bci_pdf.columns) - 

 NameError : name 'FeatureUnionDF' is not defined

In [23]:
bci_fea_tgt_pdf = fea_tgt_pipe.fit_transform(bci_pdf)
bci_fea_tgt_pdf.info()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1251867> in <module> () 
 ----> 1 bci_fea_tgt_pdf = fea_tgt_pipe . fit_transform ( bci_pdf ) 
 2 bci_fea_tgt_pdf . info ( ) 

 NameError : name 'fea_tgt_pipe' is not defined

Create a function to create train and test datasets from 
- a start date for the test set, and 
- the features and target of the preprocessed dataframe produced immediately above.

In [25]:
def get_ts_train_test_split(x,y,start_test=None):
  return (x.loc[:start_test ], y.loc[:start_test ],
          x.loc[ start_test:], y.loc[ start_test:]
         )

Now get the train and test datasets (for features and target).

In [27]:
import pandas as pd
(train_bci_fea_pdf, train_bci_tgt_ser,
  test_bci_fea_pdf,  test_bci_tgt_ser
) = get_ts_train_test_split(x=bci_fea_tgt_pdf.set_index('datetime').drop('target',axis=1), 
                            y=bci_fea_tgt_pdf.set_index('datetime').target,
                            start_test=pd.to_datetime('2018-01-01')
                           )
(train_bci_fea_pdf.shape, train_bci_tgt_ser.shape,
  test_bci_fea_pdf.shape,  test_bci_tgt_ser.shape
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1251875> in <module> () 
 2 (train_bci_fea_pdf, train_bci_tgt_ser,
 3 test_bci_fea_pdf , test_bci_tgt_ser 
 ----> 4 ) = get_ts_train_test_split ( x = bci_fea_tgt_pdf . set_index ( 'datetime' ) . drop ( 'target' , axis = 1 ) , 
 5 y = bci_fea_tgt_pdf . set_index ( 'datetime' ) . target , 
 6 start_test = pd . to_datetime ( '2018-01-01' ) 

 NameError : name 'bci_fea_tgt_pdf' is not defined

## 4. Estimator pipeline

Define a transformer class `SelectKBestDF` which wraps the `SelectKBest` class from Scikit-learn.

Notice:
- the `score_func_str` init parameter expects a string
- the use of the `super` function to call methods from the wrapped class
- the use of `self` to access the attributes of the wrapped class

In [30]:
from sklearn.feature_selection import SelectKBest
from sklearn.base              import BaseEstimator,TransformerMixin
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

class SelectKBestDF(SelectKBest):
  import numpy as np
  
  def __init__(self, k=10, score_func_str=None):
    if score_func_str is None                    : score_func = f_regression
    if score_func_str == 'f_regression'          : score_func = f_regression
    if score_func_str == 'mutual_info_regression': score_func = mutual_info_regression
    super().__init__(k=k, score_func=score_func)

  def fit(self, X, y=None):
    super().fit(X=X,y=y)
    scores_ = self.scores_
    pvalues_ = self.pvalues_

    var_ndx_sorted    = self.scores_.argsort()[::-1][:self.k]
    self.columns_keep = [X.columns[i] for i in var_ndx_sorted]
    self.columns_drop = list(set(X.columns)-set(self.columns_keep))
    return self
  
  def transform(self, X):
    return pd.DataFrame(data=super().transform(X),
                        columns=self.columns_keep)

Create a pipeline with the wrapped transformer class `SelectKBestDF` and the estimator `LinearRegression`.

In [32]:
from sklearn.pipeline          import Pipeline, FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model      import LinearRegression

est_pipe = Pipeline([
  ('skb', SelectKBestDF()),
  ('lrg', LinearRegression())
])

There are two ways to use this estimator pipeline.
1. Fit the feature and target train datasets to this estimator and make predictions on the test feature dataset. 
2. Run `GridSearchCV` with the pipeline and a parameter grid. Then compare results for different sets of hyperparameters.

Both ways are demonstrated below.

The first way is demonstrated in the following code cell.

The pipeline `est_pipe` was created with default parameters. The `set_params` method can be used to set parameters. This is demonstated in the following code cell, but will be useful further below.

In [36]:
est_pipe.set_params(**{'skb__k':1,
                       'skb__score_func_str':'f_regression'
                      })
est_pipe.fit(X=train_bci_fea_pdf, 
             y=train_bci_tgt_ser)
est_pipe.predict(X=test_bci_fea_pdf)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1251888> in <module> () 
 2 'skb__score_func_str' : 'f_regression' 
 3 })
 ----> 4 est_pipe.fit(X=train_bci_fea_pdf, 
 5 y=train_bci_tgt_ser)
 6 est_pipe . predict ( X = test_bci_fea_pdf ) 

 NameError : name 'train_bci_fea_pdf' is not defined

The second way is demonstated in the following code cell where 
- a parameter grid is created, and 
- a grid search object is created from the pipeline and parameter grid

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics         import make_scorer, r2_score

parameters = {'skb__k'             : [10, 7, 4, 1],
              'skb__score_func_str': ['f_regression', 'mutual_info_regression']
              }

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                            param_grid=parameters,
                            scoring=make_scorer(r2_score),
                            cv=3, 
                            iid=False,
                            return_train_score=False
                           )

Continuing wih the second way, fit the train and test datasets to the grid search object.

In [40]:
est_grid_obj.fit(X=train_bci_fea_pdf,
                 y=train_bci_tgt_ser
                )

Out[ 26 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('skb', SelectKBestDF(k=10, score_func_str=None)), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'skb__score_func_str': ['f_regression', 'mutual_info_regression'], 'skb__k': [10, 7, 4, 1]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring=make_scorer(r2_score), verbose=0)

Display the results. The second code cell below produces more readable and workable results by converting the dictionary to a dataframe.

In [42]:
est_grid_obj.cv_results_

Out[ 27 ]: 
{'param_skb__k': masked_array(data=[10, 10, 7, 7, 4, 4, 1, 1],
 mask=[False, False, False, False, False, False, False, False],
 fill_value='?',
 dtype=object),
 'std_fit_time': array([3.12556381e-04, 1.25453995e-04, 4.20170916e-03, 8.73326637e-04,
 9.37807358e-05, 1.00552044e-04, 1.88524365e-04, 1.10260509e-04]),
 'std_test_score': array([0.02737385, 0.02737385, 0.09497093, 0.09497093, 0.12179942,
 0.12179942, 0.03493628, 0.03493628]),
 'split1_test_score': array([0.82855655, 0.82855655, 0.83310364, 0.83310364, 0.81681635,
 0.81681635, 0.83345716, 0.83345716]),
 'param_skb__score_func_str': masked_array(data=['f_regression', 'mutual_info_regression',
 'f_regression', 'mutual_info_regression',
 'f_regression', 'mutual_info_regression',
 'f_regression', 'mutual_info_regression'],
 mask=[False, False, False, False, False, False, False, False],
 fill_value='?',
 dtype=object),
 'split0_test_score': array([0.8806512 , 0.8806512 , 0.87663135, 0.87663135, 0.60387001,
 0.60387001, 0.74807844, 0.74807844]),
 'params': [{'skb__score_func_str': 'f_regression', 'skb__k': 10},
 {'skb__score_func_str': 'mutual_info_regression', 'skb__k': 10},
 {'skb__score_func_str': 'f_regression', 'skb__k': 7},
 {'skb__score_func_str': 'mutual_info_regression', 'skb__k': 7},
 {'skb__score_func_str': 'f_regression', 'skb__k': 4},
 {'skb__score_func_str': 'mutual_info_regression', 'skb__k': 4},
 {'skb__score_func_str': 'f_regression', 'skb__k': 1},
 {'skb__score_func_str': 'mutual_info_regression', 'skb__k': 1}],
 'rank_test_score': array([1, 1, 5, 5, 7, 7, 3, 3], dtype=int32),
 'std_score_time': array([3.91012931e-05, 6.33417139e-05, 3.59236666e-05, 7.88905747e-06,
 2.03652207e-05, 3.03090775e-05, 1.27785834e-05, 1.72003211e-05]),
 'mean_fit_time': array([0.00779923, 0.00752505, 0.01030525, 0.00795984, 0.00723251,
 0.007224 , 0.00719603, 0.00734599]),
 'split2_test_score': array([0.89116322, 0.89116322, 0.65696183, 0.65696183, 0.52937817,
 0.52937817, 0.78573774, 0.78573774]),
 'mean_test_score': array([0.86679032, 0.86679032, 0.78889894, 0.78889894, 0.65002151,
 0.65002151, 0.78909111, 0.78909111]),
 'mean_score_time': array([0.00329963, 0.00330218, 0.00321166, 0.00318273, 0.00316779,
 0.00318162, 0.00315261, 0.00315094])}

A pandas dataframe will be easier to analyze. In addition, the columns can be easily sorted in the table below.

In [44]:
display_pdf(est_grid_results_pdf(est_grid_obj))

mean_test_score,param_skb__k,param_skb__score_func_str,rank_test_score,split0_test_score,split1_test_score,split2_test_score,std_test_score
0.8667903232663242,10,f_regression,1,0.8806511977723495,0.8285565527757559,0.8911632192508674,0.027373850701418154
0.8667903232663242,10,mutual_info_regression,1,0.8806511977723495,0.8285565527757559,0.8911632192508674,0.027373850701418154
0.7888989393527116,7,f_regression,5,0.8766313548607985,0.8331036359174122,0.656961827279924,0.09497093146742
0.7888989393527116,7,mutual_info_regression,5,0.8766313548607985,0.8331036359174122,0.656961827279924,0.09497093146742
0.6500215104731023,4,f_regression,7,0.6038700140048274,0.8168163507428787,0.5293781666716006,0.12179941949846443
0.6500215104731023,4,mutual_info_regression,7,0.6038700140048274,0.8168163507428787,0.5293781666716006,0.12179941949846443
0.7890911125079342,1,f_regression,3,0.748078442220349,0.833457159617723,0.7857377356857306,0.03493627721236608
0.7890911125079342,1,mutual_info_regression,3,0.748078442220349,0.833457159617723,0.7857377356857306,0.03493627721236608


Notice that a single variable is nearly as good in making predictions as 10 variables with respect to the R squared metric.

## 5. Inspect pipeline attributes

After inspecting a dataframe of results, you may want to inspect the attributes of a transformer or the estimator contained in a pipeline used by `GridSearchCV`. This section demonstrates this process. 

For instance, we inspect the attributes determined by `skb__k=1` and `skb__score_func_str='f_regression'`.

In [48]:
est_pipe.set_params(**{'skb__k':1,
                       'skb__score_func_str':'f_regression'
                      })

Out[ 20 ]: 
Pipeline(memory=None,
 steps=[('skb', SelectKBestDF(k=1, score_func_str='f_regression')), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [49]:
est_pipe.fit(X=train_bci_fea_pdf,
             y=train_bci_tgt_ser
            )

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1251901> in <module> () 
 ----> 1 est_pipe.fit(X=train_bci_fea_pdf,
 2 y = train_bci_tgt_ser 
 3 )

 NameError : name 'train_bci_fea_pdf' is not defined

In [50]:
est_pipe.named_steps['skb'].columns_keep

Out[ 48 ]: ['bci_lag3']

In [51]:
est_pipe.named_steps['lrg'].coef_

Out[ 49 ]: array([8.02789991])

## 6. Compare `SelectKBest` and `SelectKBestDF`

In [53]:
skb_df = SelectKBestDF(k=4,score_func_str='f_regression')
skb_df \
  .fit(X=train_bci_fea_pdf,
       y=train_bci_tgt_ser) 
skb_df \
  .transform(X=train_bci_fea_pdf) \
  .head()

Out[ 29 ]: 
 bci_lag3 c7_lag3 c5_lag3 p3a_03_lag3
0 12.529 10503.0 3390.0 15.718
1 12.379 10412.0 3387.0 15.705
2 12.163 10303.0 3405.0 15.923
3 12.421 10452.0 3529.0 16.286
4 13.325 10805.0 3697.0 16.486

In [54]:
skb_df.scores_

Out[ 31 ]: 
array([1.06403365e+01, 5.96626475e+00, 7.67694090e+02, 9.30246264e+02,
 5.18981039e+02, 3.56706070e+01, 3.56706070e+01, 7.35682654e+01,
 5.96626475e+00, 1.69926472e+01, 4.12966831e+02, 3.56706070e+01,
 4.12966831e+02, 1.65432415e+03, 7.35682654e+01, 2.18690002e+03,
 7.35682654e+01, 1.65432415e+03, 7.35682654e+01, 7.63393280e+03,
 2.45968410e+03, 3.56706070e+01, 5.96626475e+00, 5.96626475e+00,
 9.69720046e-01, 2.88249262e+02, 2.53393435e-02, 2.89327150e+02,
 5.22350686e-01, 2.85999848e+02])

In [55]:
np.set_printoptions(suppress=True)
skb = SelectKBest(k=4,score_func=f_regression)
skb \
  .fit(X=train_bci_fea_pdf,
       y=train_bci_tgt_ser) 
skb \
  .transform(X=train_bci_fea_pdf)[:5,:]

Out[ 32 ]: 
array([[ 12.529, 10503. , 3390. , 15.718],
 [ 12.379, 10412. , 3387. , 15.705],
 [ 12.163, 10303. , 3405. , 15.923],
 [ 12.421, 10452. , 3529. , 16.286],
 [ 13.325, 10805. , 3697. , 16.486]])

In [56]:
skb.scores_

Out[ 33 ]: 
array([ 10.64033653, 5.96626475, 767.69408953, 930.2462643 ,
 518.98103898, 35.67060696, 35.67060696, 73.56826544,
 5.96626475, 16.99264722, 412.96683135, 35.67060696,
 412.96683135, 1654.32414749, 73.56826544, 2186.90001717,
 73.56826544, 1654.32414749, 73.56826544, 7633.93279828,
 2459.68409842, 35.67060696, 5.96626475, 5.96626475,
 0.96972005, 288.2492624 , 0.02533934, 289.32714999,
 0.52235069, 285.99984767])

__The End__